# Test loading MulVal results from XSB into Python

py-xsb can be installed with `pip install py-xsb`



In [1]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%

In [2]:
!pip freeze

absl-py==0.9.0
alabaster==0.7.12
antlr4-python3-runtime==4.7.2
astor==0.8.1
attrs==19.3.0
azure-common==1.1.25
azure-nspkg==3.0.2
azure-storage==0.20.3
Babel==2.8.0
backcall==0.1.0
bibtexparser==1.1.0
bleach==3.1.0
blinker==1.4
bokeh==1.4.0
boto==2.49.0
cachetools==4.0.0
certifi==2019.11.28
cffi==1.14.0
chardet==3.0.4
ciso8601==2.1.3
click==7.1.1
contextlib2==0.6.0.post1
cowsay==2.0.3
cryptography==2.8
cycler==0.10.0
Cython==0.29.15
decorator==4.4.1
defusedxml==0.6.0
docutils==0.16
entrypoints==0.3
feedformatter==0.5
feedparser==5.2.1
flake8==3.7.9
Flask==1.1.1
Flask-Login==0.2.11
Flask-PyMongo==2.3.0
freezegun==0.3.15
future==0.18.2
gast==0.2.2
gcs-oauth2-boto-plugin==2.5
get==2019.4.13
google-api-core==1.16.0
google-auth==1.11.2
google-auth-oauthlib==0.4.1
google-cloud-bigquery==1.24.0
google-cloud-core==1.3.0
google-pasta==0.1.8
google-reauth==0.1.0
google-resumable-media==0.5.0
googleapis-common-protos==1.51.0
gprof2dot==2019.11.30
gprof2dot-magic==0.4
graphviz==0.13.2
grpcio==1.27

In [3]:

import os
import platform
import sys

import jupyter_core

sys.path.append('..')

# clone xsb from http://xsb.sourceforge.net/
# build xsb with `./configure & ./makexsb * ./makexsb dynmodule`

# pip install py-xsb of build from https://github.com/gooofy/py-xsb
from pyxsb import pyxsb_start_session, pyxsb_end_session, pyxsb_command, \
                  pyxsb_query, XSBFunctor, XSBVariable, xsb_to_json, json_to_xsb


## Set global vars here

In [4]:

BASE_DIR = '/opt/projects/diss/jupyter_nbs/mine/'
WORKING_DIR = 'data/test_001'


# to determine the location of the XSB arch dir on your machine, run this query in XSB:
# xsb_configuration:xsb_configuration(config_dir,Dir).
# XSB_ARCH_DIR = None
XSB_ARCH_DIR = '/opt/apps/xsb/XSB/config/x86_64-unknown-linux-gnu'
#print(jupyter_core.paths.jupyter_runtime_dir())
#print(JUPYTER_RUNTIME_DIR)
#jupyter_path()
#jupyter_core.paths.jupyter_path()
jupyter_core.paths.jupyter_runtime_dir()

'/home/toor/.local/share/jupyter/runtime'

## Set CWD here

In [5]:
os.chdir(BASE_DIR + WORKING_DIR)
%pwd

'/opt/projects/diss/jupyter_nbs/mine/data/test_001'

## Test XSB bridge

In [6]:

ft_txt = """
% prolog

child(anne,bridget).
child(bridget,caroline).
child(caroline,donna).
child(donna,emily).

descend(X,Y) :- child(X,Y).
descend(X,Y) :- child(X,Z), descend(Z,Y).

label('günter', "Günter").
label(anne, "Anne").
label(bridget, "Bridget").
label(caroline, "Caroline").
label(donna, "Donna").
"""

### demo using the high level XSB interface

In [7]:

# demo using the high level XSB interface
pyxsb_start_session(XSB_ARCH_DIR)

file_name = 'ft.P'
'''
w  write mode
r  read mode
a  append mode

w+  create file if it doesn't exist and open it in (over)write mode
    [it overwrites the file if it already exists]
r+  open an existing file in read+write mode
a+  create file if it doesn't exist and open it in append mode
'''
f = open(file_name, 'w+')  # open file in append mode
f.write(ft_txt)
f.close()

pyxsb_command('consult(ft).')
print("=================")
for row in pyxsb_query('label(X, L).'):
    print(u"label of %s is %s" % (row[0], row[1]))
print("=================")
# structured XSB* interface
for row in pyxsb_query(XSBFunctor('descend', [XSBVariable('X'), XSBVariable('Y')])):
    print(u"decendant of %s is %s" % (row[0], row[1]))
print("=================")
for row in pyxsb_query(u'A = 1, B = 0.5, C = "hello", D = yes, E = foo(bar), F = [1.1,2.2], G = \'günter\'.'):

    for i, r in enumerate(row):
        print(u"#%d: %-10s (type: %-20s, class: %-20s)" % (i, r, type(r), r.__class__))

    js = xsb_to_json(row)
    print("json: %s" % js)

    row2 = json_to_xsb(js)
print("=================")
pyxsb_query('catch(abort,Exception,true).')
try:
    # throw has no effect
    pyxsb_query("writeln('Trying to throw up:'), throw('test throw').")
except:
    print('Test throw -- caught')
print("=================")
# testing abort
try:
    pyxsb_query("abort('Test abort').")
except:
    print('Test abort -- caught')
print("=================")
print('Testing complex explicit module names')
pyxsb_command('assert(ppp(simplemod:abcde(p,q,r))).')
pyxsb_command('assert(ppp(mod(arg1,f(arg2)):abc(p,q,r))).')
pyxsb_command('assert(ppp(mod2(arg,g(arg)):cde(p,q,r))).')
for row in pyxsb_query("ppp(X)."):
    print('answer: ', row[0])
print("=================")
pyxsb_end_session()

label of 'günter' is "Günter"
label of 'anne' is "Anne"
label of 'bridget' is "Bridget"
label of 'caroline' is "Caroline"
label of 'donna' is "Donna"
decendant of 'anne' is 'bridget'
decendant of 'bridget' is 'caroline'
decendant of 'caroline' is 'donna'
decendant of 'donna' is 'emily'
decendant of 'anne' is 'caroline'
decendant of 'anne' is 'donna'
decendant of 'anne' is 'emily'
decendant of 'bridget' is 'donna'
decendant of 'bridget' is 'emily'
decendant of 'caroline' is 'emily'
#0: 1          (type: <class 'int'>       , class: <class 'int'>       )
#1: 0.5        (type: <class 'float'>     , class: <class 'float'>     )
#2: "hello"    (type: <class 'pyxsb.XSBString'>, class: <class 'pyxsb.XSBString'>)
#3: 'yes'      (type: <class 'pyxsb.XSBAtom'>, class: <class 'pyxsb.XSBAtom'>)
#4: foo('bar') (type: <class 'pyxsb.XSBFunctor'>, class: <class 'pyxsb.XSBFunctor'>)
#5: [1.1, 2.2] (type: <class 'list'>      , class: <class 'list'>      )
#6: 'günter'   (type: <class 'pyxsb.XSBAtom'>, c

### demo using the low level XSB interface

In [8]:
ctest_txt = """
%prolog
% this is just to test that C calling XSB works with the preprocessor
:- compiler_options([xpp_on]).
#include "char_defs.h"

p(100,aa,bb).
p(300,first,second).
p(400,wrong,one).
p(300,third,fourth).
p(X,Y,Z) :- repeat, 
	writeln('Enter a term of the form p(300,atom1,atom2). (or end_of_file.)'),
	write(': '),
	read(T), 
	(T=end_of_file -> 
		!,fail
	;
		true
	),
	T=p(X,Y,Z).
"""

In [9]:
# demo using the low level XSB interface 

file_name = 'ctest.P'
'''
w  write mode
r  read mode
a  append mode

w+  create file if it doesn't exist and open it in (over)write mode
    [it overwrites the file if it already exists]
r+  open an existing file in read+write mode
a+  create file if it doesn't exist and open it in append mode
'''
f = open(file_name, 'w+')  # open file in append mode
f.write(ctest_txt)
f.close()

pyxsb_start_session(XSB_ARCH_DIR)

# should be after start_xsb_session so that imported vars will be updated
from pyxsb import *

c2p_functor(b"consult", 1, reg_term(1))
c2p_string(b"ctest",p2p_arg(reg_term(1),1))
if xsb_command():
    raise Exception ("Error consulting ctest")

if xsb_command_string(b"consult(basics)."):
    raise Exception ("Error (string) consulting basics.")

# Create the query p(300,X,Y) and send it.
c2p_functor(b"p",3,reg_term(1))
c2p_int(300,p2p_arg(reg_term(1),1))

rcode = xsb_query()

# Print out answer and retrieve next one.
while not rcode:

    if not is_string(p2p_arg(reg_term(2),1)) and is_string(p2p_arg(reg_term(2),2)):
        print ("2nd and 3rd subfields must be atoms")
    else:
        print ("Answer: %d, %s(%s), %s(%s)" % ( p2c_int(p2p_arg(reg_term(1),1)),
                                               p2c_string(p2p_arg(reg_term(1),2)),
                                               xsb_var_string(1),
                                               p2c_string(p2p_arg(reg_term(1),3)),
                                               xsb_var_string(2)))
    rcode = xsb_next()

# Create the string query p(300,X,Y) and send it, use higher-level routines.

xsb_make_vars(3)
xsb_set_var_int(300,1)
rcode = xsb_query_string(b"p(X,Y,Z).")

# Print out answer and retrieve next one.
while not rcode:
    if not is_string(p2p_arg(reg_term(2),2)) and is_string(p2p_arg(reg_term(2),3)):
        print ("2nd and 3rd subfields must be atoms")
    else:
        print ("Answer: %d, %s, %s" % (xsb_var_int(1),
                                      xsb_var_string(2),
                                      xsb_var_string(3)))
    rcode = xsb_next()



pyxsb_query('catch(abort,Exception,true).')

pyxsb_command('consult(ft).')


pyxsb_end_session()

Answer: 300, b'first'(first), b'second'(second)
Answer: 300, b'third'(third), b'fourth'(fourth)
Answer: 300, first, second
Answer: 300, third, fourth


In [10]:
# pyxsb_query('catch(abort,Exception,true).')

In [11]:
pyxsb_start_session(XSB_ARCH_DIR)


pyxsb_end_session()

In [12]:
pyxsb_start_session(XSB_ARCH_DIR)
pyxsb_command('consult(run).')
pyxsb_end_session()

## MulVal KB Queries for XSB

In [13]:
pyxsb_start_session(XSB_ARCH_DIR)

print('--------------------')
for row in pyxsb_query('hacl(A, B, C, D).'):
    print(u"hacl %s %s %s %s" % (row[0], row[1], row[2], row[3]))

print('--------------------')

for row in pyxsb_query('vulProperty(A,B,C).'):
    print(u"vulprop %s %s %s" % (row[0], row[1], row[2]))


print('---------attackGoal-----------')
print(pyxsb_query('findall(attackGoal(A), attackGoal(A), L), list_apply(L,write_clause_to_stdout).'))

print('---------attackerLocated-----------')
print(pyxsb_query('findall(attackerLocated(A), attackerLocated(A), L), list_apply(L,write_clause_to_stdout).'))


    
print('---------vulProperty-----------')
print(pyxsb_query('findall(vulProperty(A,B,C),vulProperty(A,B,C),L),list_apply(L,write_clause_to_stdout).'))

print('----------vulExists----------')
print(pyxsb_query('findall(vulExists(A,B,C),vulExists(A,B,C),L),list_apply(L,write_clause_to_stdout).'))

print('---------cvss-----------')
print(pyxsb_query('findall(cvss(CVE, AC),cvss(CVE, AC),L),list_apply(L,write_clause_to_stdout).'))


print('----------hacl----------')
print(pyxsb_query('findall(hacl(A, B, C, D),hacl(A, B, C, D),L),list_apply(L,write_clause_to_stdout).'))

print('---------networkServiceInfo-----------')
print(pyxsb_query('findall(networkServiceInfo(Host, Program, someProtocol, somePort, someUser), networkServiceInfo(Host, Program, someProtocol, somePort, someUser), L), list_apply(L,write_clause_to_stdout).'))

print('---------hasAccount-----------')
print(pyxsb_query('findall(hasAccount(A,B,C), hasAccount(A,B,C), L), list_apply(L,write_clause_to_stdout).'))

print('---------inCompetent-----------')
print(pyxsb_query('findall(inCompetent(A), inCompetent(A), L), list_apply(L,write_clause_to_stdout).'))

print('---------inSubnet-----------')
print(pyxsb_query('findall(inSubnet(A, B)., inSubnet(A, B), L), list_apply(L,write_clause_to_stdout).'))

# print('============Derived===============')
# print('---------execCode-----------')
# print(pyxsb_query('findall(execCode(A, B)., execCode(A, B), L), list_apply(L,write_clause_to_stdout).'))



pyxsb_end_session()




--------------------
hacl '10.0.0.1/32' '10.0.0.2/32'  
hacl '10.0.0.2/32' '10.0.0.3/32'  
hacl '10.0.0.3/32' '10.0.0.4/32'  
hacl '10.0.0.4/32' '10.0.0.5/32' 'tcp' 21
--------------------
---------attackGoal-----------
[XSBFunctor(name=ret, module=None, args=[XSBVariable(name= ), [XSBFunctor(name=attackGoal, module=None, args=[XSBFunctor(name=execCode, module=None, args=[XSBAtom(name=10.0.0.5/32), XSBVariable(name=)])])]])]
---------attackerLocated-----------
[XSBFunctor(name=ret, module=None, args=[XSBVariable(name= 	), [XSBFunctor(name=attackerLocated, module=None, args=[XSBAtom(name=10.0.0.3/32)])]])]
---------vulProperty-----------
[XSBFunctor(name=ret, module=None, args=[XSBVariable(name= ), XSBVariable(name=), XSBVariable(name=@), XSBAtom(name=[])])]
----------vulExists----------
[XSBFunctor(name=ret, module=None, args=[XSBVariable(name=), XSBVariable(name=`), XSBVariable(name=), XSBAtom(name=[])])]
---------cvss-----------
[XSBFunctor(nam

In [14]:
%pwd

'/opt/projects/diss/jupyter_nbs/mine/data/test_001'

# MulVal Input Network Transformer

We need a representation of our network model as it is available to attack graph generation. So, just before we run mulval,  let's dump a .dot representation of current knowledge base. 

In [17]:
input_p = """attackerLocated(internet).
attackGoal(execCode(workStation,_)).

hacl(internet, webServer, tcp, 80).
hacl(webServer, _,  _, _).
hacl(fileServer, _, _, _).
hacl(workStation, _, _, _).
hacl(H,H,_,_).

/* configuration information of fileServer */
networkServiceInfo(fileServer, mountd, rpc, 100005, root).
nfsExportInfo(fileServer, '/export', _anyAccess, workStation).
nfsExportInfo(fileServer, '/export', _anyAccess, webServer).
vulExists(fileServer, vulID, mountd).
vulProperty(vulID, remoteExploit, privEscalation).
localFileProtection(fileServer, root, _, _).

/* configuration information of webServer */
vulExists(webServer, 'CAN-2002-0392', httpd).
vulProperty('CAN-2002-0392', remoteExploit, privEscalation).
networkServiceInfo(webServer , httpd, tcp , 80 , apache).

/* configuration information of workStation */
nfsMounted(workStation, '/usr/local/share', fileServer, '/export', read).
"""

## Setup py_mulval project path, imports, setup, etc

In [51]:
py_mulval_path = r'/opt/projects/diss/py-mulval/src'
sys.path.append(py_mulval_path)

notebook_url = 'localhost:8888'

import py_mulval.py_mulval
import py_mulval.metrics
import py_mulval.attack_graph as attack_graph
from py_mulval.attack_graph import AttackGraph

from py_mulval import flags
FLAGS = flags.FLAGS
FLAGS([sys.argv[0]]) # dont expect cli args from jupyter

from absl import app
from absl import flags
import logging
import os
import pathlib
import re
import subprocess
import sys
import tempfile
import uuid

from py_mulval import boromir # importing flags only
from py_mulval import flag_util
from py_mulval import genTransMatrix
from py_mulval import log_util
from py_mulval import publisher
from py_mulval import py_mulval
from py_mulval import sample

FLAGS = flags.FLAGS

SEP = os.path.sep

"""
Usage: graph_gen.sh [-r|--rule rulefile]
            [-a|--additional additional_rulefile]
    [-c|--constraint constraint_file]
    [-g|--goal goal]
    [-d|--dynamic dynamic_file]
    [-p]
    [-s|--sat]
    [-t|--t trace_option]
    [-tr|--trim]
    [-v|--visualize [--arclabel] [--reverse]]
            [--cvss]
        [-h|--help]
        [attack_graph_options]
        input_file
"""

'\nUsage: graph_gen.sh [-r|--rule rulefile]\n            [-a|--additional additional_rulefile]\n    [-c|--constraint constraint_file]\n    [-g|--goal goal]\n    [-d|--dynamic dynamic_file]\n    [-p]\n    [-s|--sat]\n    [-t|--t trace_option]\n    [-tr|--trim]\n    [-v|--visualize [--arclabel] [--reverse]]\n            [--cvss]\n        [-h|--help]\n        [attack_graph_options]\n        input_file\n'

## utility functions for working with mulval. 

these flags are imported from original source, so don't redefine them  here

In [43]:

# flags.DEFINE_string('helpmatch', None, 'help flag')
# flags.DEFINE_string('helpmatchmd', None, 'help markdown flag')

#####
# Mulpy Flags
#####
# # flags.DEFINE_string('run_uri', None, 'Unique ID for this run.')
# flags.DEFINE_string('base_dir', None, 'Working dir. Default: /tmp/<run_uri>/')
# flags.DEFINE_string('data_dir', None, 'Directory to find scripts, ymls, etc.')
# flags.DEFINE_string('models_dir', None, 'Prefix model files with this path.')
# flags.DEFINE_string('rules_dir', None, 'Prefix rule files with this path.')
# flags.DEFINE_string('output_dir', None, 'Prefix output files with this path.')


def _GetTempDir():
  return tempfile.gettempdir()


def _SetBaseDir():
  if not FLAGS.base_dir:
    FLAGS.base_dir = SEP.join((_GetTempDir(), 'mulpy', 'runs'))
  FLAGS.base_dir = SEP.join((FLAGS.base_dir, FLAGS.run_uri))
  logging.info('Base directory set to: {}'.format(FLAGS.base_dir))
  if not pathlib.Path(FLAGS.base_dir).exists():
    pathlib.Path(FLAGS.base_dir).mkdir(parents=True, exist_ok=True)


def _SetModelsDir():
  if not FLAGS.models_dir:
    FLAGS.models_dir = SEP.join((FLAGS.base_dir, 'models'))
  logging.info('Models directory set to: {}'.format(FLAGS.models_dir))
  if not pathlib.Path(FLAGS.models_dir).exists():
    pathlib.Path(FLAGS.models_dir).mkdir(parents=True, exist_ok=True)

def _SetOutputDir():
  if not FLAGS.output_dir:
    FLAGS.output_dir = SEP.join((FLAGS.base_dir, 'output'))
  logging.info('Output directory set to: {}'.format(FLAGS.output_dir))
  if not pathlib.Path(FLAGS.output_dir).exists():
    pathlib.Path(FLAGS.output_dir).mkdir(parents=True, exist_ok=True)

#
# def _SetRulesDir():
#   if not FLAGS.rules_dir:
#     FLAGS.models_dir = SEP.join((FLAGS.base_dir, 'models'))
#   logging.info('Models directory set to: {}'.format(FLAGS.models_dir))
#   if not pathlib.Path(FLAGS.models_dir).exists():
#     pathlib.Path(FLAGS.models_dir).mkdir(parents=True, exist_ok=True)


def writeFile(file_name, file_text, mode='w+'):
  """
  w  write mode
  r  read mode
  a  append mode
  w+  create file if it doesn't exist and open it in (over)write mode
      [it overwrites the file if it already exists]
  r+  open an existing file in read+write mode
  a+  create file if it doesn't exist and open it in append mode
  """
  with open(file_name, mode) as file:
    file.write(file_text)


def _ParseFlags(argv=sys.argv):
  """Parses the command-line flags."""
  try:
    argv = FLAGS(argv)
    logging.debug('Parsed command line flags: {}'.format(FLAGS.input_file))
  except flags.Error as e:
    logging.error(e)
    sys.exit(1)


def InitRunID():
  FLAGS.run_uri = str(uuid.uuid4())[-8:]
  logging.info('Run ID initialized: {}'.format(FLAGS.run_uri))


def SetupMulpy():
  logging.info('Setting up Mulpy environment...')
  InitRunID()
  _SetBaseDir()
  # setup logging once we have a run_uri and base dir
  log_util.ConfigureLogging(logging.DEBUG,
                            SEP.join((FLAGS.base_dir, 'cat-dog.log')),
                            FLAGS.run_uri)
  # try to get cli into main log... not always trustworthy
  logging.info('running command line: %s' % ' '.join(sys.argv))
  _SetModelsDir()
  # _SetRulesDir() # if no rules_dir just use default set
  _SetOutputDir()
  # Copy model or example into base_dir
  if pathlib.Path(SEP.join((FLAGS.models_dir, FLAGS.input_file))).exists():
    input_p = pathlib.Path(
      SEP.join((FLAGS.models_dir, FLAGS.input_file))).read_text()
  else:
    input_p = """attackerLocated(internet).
  attackGoal(execCode(workStation,_)).

  hacl(internet, webServer, tcp, 80).
  hacl(webServer, _,  _, _).
  hacl(fileServer, _, _, _).
  hacl(workStation, _, _, _).
  hacl(H,H,_,_).

  /* configuration information of fileServer */
  networkServiceInfo(fileServer, mountd, rpc, 100005, root).
  nfsExportInfo(fileServer, '/export', _anyAccess, workStation).
  nfsExportInfo(fileServer, '/export', _anyAccess, webServer).
  vulExists(fileServer, vulID, mountd).
  vulProperty(vulID, remoteExploit, privEscalation).
  localFileProtection(fileServer, root, _, _).

  /* configuration information of webServer */
  vulExists(webServer, 'CAN-2002-0392', httpd).
  vulProperty('CAN-2002-0392', remoteExploit, privEscalation).
  networkServiceInfo(webServer , httpd, tcp , 80 , apache).

  /* configuration information of workStation */
  nfsMounted(workStation, '/usr/local/share', fileServer, '/export', read).
  """
  # write input file to working directory if it doesn't exist
  outfile = SEP.join((FLAGS.base_dir, FLAGS.input_file))
  logging.debug(('creating input file: %s') % outfile)
  if not pathlib.Path(outfile).exists():
    with open(outfile, 'w+') as file:
      file.write(input_p)
    logging.debug(('creating input file: %s') % outfile)


## _RunMulval funtion here

In [44]:

def _RunMulVal():

  mulval_args = {}
  if FLAGS.input_file and FLAGS.models_dir:
    mulval_args['input_file'] = SEP.join((FLAGS.models_dir, FLAGS.input_file))
  if not FLAGS.input_file and FLAGS.models_dir:
    pass # @TODO handle all models in dir


  #####
  ## graph_gen.sh
  ####
  gg = py_mulval.graph_gen(**mulval_args)
  gg.graph_gen()

  #####
  ## attack_graph.cpp
  ####
  ag_args = {}
  ag = py_mulval.attack_graph(**ag_args)

  ag_text = ag.attack_graph().decode('UTF-8')
  gg.writeFile(FLAGS.base_dir + '/AttackGraph.txt', ag_text)

  verts = ''
  arcs = ''
  for line in ag_text.splitlines():
    if re.search(r'AND|OR|LEAF', line):
      verts += line + '\n'
    else:
      arcs += line + '\n'

  # logging.info('arcs: %s' % arcs)
  # logging.info('verts: %s' % verts)

  gg.writeFile(FLAGS.base_dir + '/ARCS.CSV', arcs)
  gg.writeFile(FLAGS.base_dir + '/VERTICES.CSV', verts)

  ag.render()



## Start Testing here

In [55]:
log_util.ConfigureBasicLogging()
logging.info('Basic Logging configured.')
# SetupMulpy()
# _ParseFlags()

SetupMulpy()

# _RunMulVal()

2020-04-28 14:15:26,310 e32ed4c9 MainThread INFO     Basic Logging configured.
2020-04-28 14:15:26,311 e32ed4c9 MainThread INFO     Setting up Mulpy environment...
2020-04-28 14:15:26,312 e32ed4c9 MainThread INFO     Run ID initialized: 61189139
2020-04-28 14:15:26,313 e32ed4c9 MainThread INFO     Base directory set to: /tmp/mulpy/runs/e6636fa6/736b3ae5/efc40c21/70386a2e/e32ed4c9/61189139
2020-04-28 14:15:26,313 61189139 MainThread INFO     Verbose logging to: /tmp/mulpy/runs/e6636fa6/736b3ae5/efc40c21/70386a2e/e32ed4c9/61189139/cat-dog.log
2020-04-28 14:15:26,314 61189139 MainThread INFO     running command line: /opt/projects/diss/jupyter_nbs/py36/lib/python3.6/site-packages/ipykernel_launcher.py -f /home/toor/.local/share/jupyter/runtime/kernel-f06aee89-5c1b-4307-a08d-29fe6bd0dff3.json
2020-04-28 14:15:26,315 61189139 MainThread INFO     Models directory set to: /tmp/mulpy/runs/e6636fa6/models
2020-04-28 14:15:26,316 61189139 MainThread INFO     Output directory set to: /tmp/mulpy/r

TypeError: sequence item 1: expected str instance, NoneType found

In [54]:
# print(FLAGS)
%tb

SystemExit: 1

In [35]:
%tb

SystemExit: 1